In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('vibemeter_dataset.csv')
df.head()

,Employee_ID,Response_Date,Vibe_Score
0,EMP0454,01-01-2023,3
1,EMP0071,02-01-2023,1
2,EMP0096,03-01-2023,4
3,EMP0206,04-01-2023,5
4,EMP0074,05-01-2023,3


In [ ]:
df['Vibe_Score'].max()

5

In [ ]:
df.sort_values(by=["Employee_ID", "Response_Date"], ascending=[True, False], inplace=True)
df.head()

,Employee_ID,Response_Date,Vibe_Score
28,EMP0002,29-01-2023,1
363,EMP0003,30-12-2023,1
406,EMP0003,11-02-2024,3
414,EMP0004,19-02-2024,4
117,EMP0005,28-04-2023,5


In [ ]:
df["Response_Date"] = pd.to_datetime(df["Response_Date"], errors='coerce')
df.sort_values(by=["Employee_ID", "Response_Date"], ascending=[True, False], inplace=True)

decay_factor = 1 / 5

df["Days_Since"] = df.groupby("Employee_ID")["Response_Date"].transform(lambda x: (x.max() - x).dt.days)

df["Weight"] = np.exp(-decay_factor * df["Days_Since"])

df.head()

<ipython-input-4-fa1fa0ca4930>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Response_Date"] = pd.to_datetime(df["Response_Date"], errors='coerce')


,Employee_ID,Response_Date,Vibe_Score,Days_Since,Weight
28,EMP0002,2023-01-29,1,0,1.000000
406,EMP0003,2024-02-11,3,0,1.000000
363,EMP0003,2023-12-30,1,43,0.000184
414,EMP0004,2024-02-19,4,0,1.000000
343,EMP0005,2023-12-10,3,0,1.000000


In [ ]:
# Compute weighted emotion score per employee
weighted_vibe = df.groupby("Employee_ID").apply(
    lambda emp: pd.Series({
        "Weighted_vibe_Score": np.sum(emp["Vibe_Score"] * emp["Weight"]) / np.sum(emp["Weight"])
    })
).reset_index()

weighted_vibe["Weighted_vibe_Score"] = weighted_vibe["Weighted_vibe_Score"].round(4)

# Display the final weighted emotion scores
weighted_vibe.head()

<ipython-input-5-9a1ef83a6afd>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_vibe = df.groupby("Employee_ID").apply(


,Employee_ID,Weighted_vibe_Score
0,EMP0002,1.0000
1,EMP0003,2.9996
2,EMP0004,4.0000
3,EMP0005,3.0000
4,EMP0006,5.0000


In [ ]:
weighted_vibe["Weighted_vibe_Score"].max()

5.0

In [ ]:
weighted_vibe.shape

(315, 2)

In [ ]:
weighted_vibe['Employee_ID'].nunique()

315

In [ ]:
output_path = "unique_employees_weighted_vibe.csv"
weighted_vibe.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")


Processed data saved to unique_employees_weighted_vibe.csv
